In [2]:
from collections import defaultdict
import numpy as np

In [453]:
# CSA helpers

def search_next_departure(stop_departures, time):
    '''
        Search the first departure after moment time in the same stop through binary search in the list of S[stop]
    '''
    start = 0
    end = len(stop_departures) - 1

    next_departure_index = -1
    while start <= end:
        mid = (start + end) // 2;

        # Search in the right side
        if (stop_departures[mid]['departure_time'] < time):
            start = mid + 1

        # Search in the left side
        else:
            next_departure_index = mid
            end = mid - 1
     
    
    # Search is always successful because of the last np.inf departure
    next_profile_entry = stop_departures[next_departure_index]
        
    return next_departure_index, next_profile_entry

def shift(vector, include_earliest_arrival=False):
    # TODO: include max for earliest arrival
    new_vector = vector.copy()
    new_vector[1:] = vector[:-1]
    new_vector[0] = np.inf
    if include_earliest_arrival:
        new_vector[-1] = min(vector[-1], vector[-2])
    return new_vector

def minimize(vector1, vector2):
    return np.minimum(vector1, vector2)

def minimize_with_exits(vector_with_exits, tau_c, connection):
    indices_to_change = np.where(tau_c < vector_with_exits[0])[0]
    
    new_arrival_times = vector_with_exits[0].copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_exits = vector_with_exits[1].copy()
    for index in indices_to_change:
        new_exits[index] = connection
    return (new_arrival_times, new_exits)
    
def minimize_with_stops(old_arrivals, old_connections, tau_c, connection):
    indices_to_change = np.where(tau_c < old_arrivals)[0]
    
    new_arrival_times = old_arrivals.copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_connections = old_connections.copy()
    for index in indices_to_change:
        new_connections[index] = connection
    return (new_arrival_times, new_connections)

def minimize_with_enter_exit_connections(old_arrivals, old_enter_connections, tau_c, connection, old_exit_connections, trip_exit_connections):
    indices_to_change = np.where(tau_c < old_arrivals)[0]
    
    new_arrival_times = old_arrivals.copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_enter_connections = old_enter_connections.copy()
    new_exit_connections = old_exit_connections.copy()
    for index in indices_to_change:
        new_enter_connections[index] = connection
        new_exit_connections[index] = trip_exit_connections[index]
        
    return (new_arrival_times, new_enter_connections, new_exit_connections)

In [454]:
def CSA(timetable, target_stop, min_departure_time, max_arrival_time, max_connections, include_earliest_arrival=False):
    '''
        Implementation of Pareto Connection Scan profile algorithm with interstop footpaths (https://arxiv.org/pdf/1703.05997.pdf)
    '''
    # timetable
    stops, connections, trips, footpaths = timetable
    
    # if including the earliest arrival, increase max connections by one
    if include_earliest_arrival:
        max_connections += 1
    
    # prepare S, T and pre-process D
    S = defaultdict(lambda: [{'departure_time': np.inf, 'arrival_times': np.ones(max_connections) * np.inf, 'enter_connections': [None] * max_connections, 'exit_connections': [None] * max_connections}]) # entries of the type       stop: [(departure_time, [arrival_time_with_1_connection, ..., arrival_time_with_max_connections], incoming_connection, [outgoing_connections])]
    T = defaultdict(lambda: {'arrival_times': np.ones(max_connections) * np.inf, 'exit_connections': [None] * max_connections})   # trip: [arrival_time_with_1_connection, ..., arrival_time_with_max_connections], incoming_connection
    D = defaultdict(lambda: np.inf)
    
    # initialize footpaths from stops to target stop
    for footpath in footpaths[target_stop]:
        f_dep_stop, f_dur = footpath
        D[f_dep_stop] = f_dur
    
    for connection in connections:
        c_dep_stop, c_arr_stop, c_dep_time, c_arr_time, c_trip = connection
        
        # Initialize exit connections of trip with the last temporal connection for this trip: after this trip, it is necessary to exit the line as it does not lead anywhere
        if T[c_trip]['exit_connections'][0] is None:
            T[c_trip]['exit_connections'] = [connection] * max_connections
        
        # PHASE 1: FIND tau_c (arrival times given this connection)
        
        # Arrival times by walk from c_arr_stop 
        tau_1 = np.ones(max_connections) * (c_arr_time if c_arr_stop == target_stop else c_arr_time + D[c_arr_stop])
        
        # Arrival times by continuing on the same trip
        tau_2 = T[c_trip]['arrival_times']
        
        # Arrival times by moving to another trip
        # TODO add walk time
        tau_3 = search_next_departure(S[c_arr_stop], c_arr_time)[1]['arrival_times']
        tau_3 = shift(tau_3, include_earliest_arrival)
        
        # Find minimum per number of changes between tau_1, tau_2 and tau_3
        tau_c = minimize(minimize(tau_1, tau_2), tau_3)

        # PHASE 2: UPDATE S
        
        # arrivals of earliest departure after c_dep_time from stop c_dep_stop
        next_departure_index, next_profile_entry = search_next_departure(S[c_dep_stop], c_dep_time)
        y = next_profile_entry['arrival_times']
        y_departure_connections = next_profile_entry['enter_connections']
        y_exit_connections = next_profile_entry['exit_connections']
        new_min_arrivals, new_enter_connections, new_exit_connections = minimize_with_enter_exit_connections(y, y_departure_connections, tau_c, connection, y_exit_connections, T[c_trip]['exit_connections'])
        
        # If the new candidate profile ix not dominated
        if not np.array_equal(y, new_min_arrivals):
            # Update for the departure stop of the connection (as if there was a self-loop in footpaths)
            S[c_dep_stop].insert(next_departure_index, {'departure_time': c_dep_time, 'arrival_times': new_min_arrivals, 'enter_connections': new_enter_connections, 'exit_connections': new_exit_connections})
            
            # Update for the footpath departure stop of all footpaths towards the departure stop of the connection
            for footpath in footpaths[c_dep_stop]:
                f_dep_stop, f_dur = footpath
                next_departure_index, next_profile_entry = search_next_departure(S[f_dep_stop], c_dep_time - f_dur)
                y = next_profile_entry['arrival_times']
                y_departure_connections = next_profile_entry['enter_connections']
                y_exit_connections = next_profile_entry['exit_connections']
                new_min_arrivals, new_enter_connections, new_exit_connections = minimize_with_enter_exit_connections(y, y_departure_connections, tau_c, connection, y_exit_connections, T[c_trip]['exit_connections'])
                S[f_dep_stop].insert(next_departure_index, {'departure_time': c_dep_time, 'arrival_times': new_min_arrivals, 'enter_connections': new_enter_connections, 'exit_connections': new_exit_connections})
        
        # PHASE 3: UPDATE T
        
        new_min_arrivals, new_exit_connections = minimize_with_stops(T[c_trip]['arrival_times'], T[c_trip]['exit_connections'], tau_c, connection)
        T[c_trip]['arrival_times'] = new_min_arrivals
        T[c_trip]['exit_connections'] = new_exit_connections
        
    return S

In [455]:
def extract_paths(S, source_stop, source_time, target_stop, changes=None):
    profiles_source_stop = S[source_stop]
    first_departure_index, _ = search_next_departure(profiles_source_stop, source_time)
    
    paths = set()
    
    for i in range(first_departure_index, len(profiles_source_stop)):
        profile_entry = profiles_source_stop[i]
        enter_exit_connections = set(zip(profile_entry['enter_connections'], profile_entry['exit_connections']))
        for current_trip in enter_exit_connections:
            enter_connection, exit_connection = current_trip
            if enter_connection is None: # If it is not possible to reach target from here in the selected number of changes
                continue       
            if enter_connection[1] == target_stop or exit_connection[1] == target_stop: # If this connection or this line is directed to the target stop 
                paths.add((enter_connection, exit_connection))
            else: # If we need to exit the line at some point after this connection
                next_source_stop = exit_connection[1]
                next_source_time = exit_connection[3]
                paths_from_next = extract_journeys(S, next_source_stop, next_source_time, target_stop, changes - 1 if changes is not None else None)
                paths_from_current = [current_trip + path for path in paths_from_next]
                paths.update(paths_from_current)

    return paths

In [456]:
class Link:
    def __init__(self, dep_stop, arr_stop, dep_time, arr_time):
        self.dep_stop = dep_stop
        self.arr_stop = arr_stop
        self.dep_time = dep_time
        self.arr_time = arr_time

    def get_dep_stop(self):
        return self.dep_stop

    def get_dep_time(self):
        return self.dep_time

    def get_arr_stop(self):
        return self.arr_stop

    def get_arr_time(self):
        return self.arr_time


class Trip(Link):
    def __init__(self, dep_stop, arr_stop, dep_time, arr_time, trip_id, maximum_delay):
        super().__init__(dep_stop, arr_stop, dep_time, arr_time)
        self.trip_id = trip_id
        self.confidence = self.compute_confidence(maximum_delay)

    def get_trip_id(self):
        return self.trip_id

    def get_confidence(self):
        return self.confidence

    def compute_confidence(self, maximum_delay):
        # TODO
        return 1.

    def __str__(self):
        return "At {}, take the line {} with arrival at time {} in {}".format(self.dep_time, self.trip_id,
                                                                              self.arr_time, self.arr_stop)


class Footpath(Link):
    def __init__(self, dep_stop, arr_stop, dep_time, arr_time):
        super().__init__(dep_stop, arr_stop, dep_time, arr_time)

    def __str__(self):
        return "At {}, walk from station {} to station {}, with expected arrival at {}.".format(self.dep_time,
                                                                                                self.dep_stop,
                                                                                                self.arr_stop,
                                                                                                self.arr_time)


class Change:
    def __init__(self, stop, previous_trip_id, following_trip_id):
        self.stop = stop
        self.previous_trip_id = previous_trip_id
        self.following_trip_id = following_trip_id

    def __str__(self):
        return "After reaching station {}, change from line {} to line {} (estimated change time of 2 minutes)".format(
            self.stop,
            self.previous_trip_id,
            self.following_trip_id)


class Journey:
    def __init__(self):
        self.links = []
        self.confidence = 1.
        self.num_connections = 0

    def __len__(self):
        return self.num_connections

    def __str__(self):
        return "\n".join([str(link) for link in self.links])

    def add_trip(self, trip):
        self.links.append(trip)
        self.num_connections += 1
        self.confidence *= trip.get_confidence()

    def add_change(self, change):
        self.links.append(change)

    def add_footpath(self, footpath):
        self.links.append(footpath)


def analyze_path(path, maximum_arrival_time, footpaths):
    '''
        From paths (list of pairs of enter_connection, exit_connections) to journeys (list of links)
    '''

    if path is None:
        return None

    journey = Journey()
    path_len = len(path) // 2

    # Add first link
    trip_start = path[0]
    trip_end = path[1]
    next_departure_time = path[2][2] if len(path) > 2 else maximum_arrival_time
    maximum_delay = next_departure_time - trip_end[3]
    trip = Trip(dep_stop=trip_start[0], arr_stop=trip_end[1], dep_time=trip_start[2], arr_time=trip_end[3],
                trip_id=trip_start[4], maximum_delay=maximum_delay)
    journey.add_trip(trip)

    # Update previous exit connection for detecting footpaths
    previous_exit_connection = path[1]

    # Add all the other links
    for i in range(1, path_len):
        trip_start = path[i * 2]
        trip_end = path[i * 2 + 1]

        # If the last arrival stop is not the following departing stations, we need to walk from one stop to the other
        if trip_start[0] != previous_exit_connection[1]:
            footpath = Footpath(previous_exit_connection[1], trip_start[0], previous_exit_connection[3],
                                footpaths[trip_start[0]][previous_exit_connection[1]])
            journey.add_footpath(footpath)

        # Otherwise, we just need to change in the same station before the new link if the departure station is not 
        # the first one 
        else:
            change = Change(trip_start[0], previous_exit_connection[4], trip_start[4])
            journey.add_change(change)

        # Finally, add the new trip after the footpath / change
        next_departure_time = path[(i + 1) * 2][2] if len(path) > (i + 1) * 2 else maximum_arrival_time
        maximum_delay = next_departure_time - trip_end[3]
        trip = Trip(dep_stop=trip_start[0], arr_stop=trip_end[1], dep_time=trip_start[2], arr_time=trip_end[3],
                    trip_id=trip_start[4], maximum_delay=maximum_delay)
        journey.add_trip(trip)

        # Update previous exit connection for detecting footpaths
        previous_exit_connection = trip_end

    return journey


In [457]:
def describe_path(path):
    path_len = len(path) // 2 
    for i in range(path_len):
        trip_start = path[i*2]
        trip_end = path[i*2+1]
        print("TRIP {0}. Enter the line at the station {1} at {2} and exit the line at the station {3} at {4}.".format(trip_start[4], trip_start[0], trip_start[2], trip_end[1], trip_end[3]))

In [463]:
def plan_route(timetable, source_stop, target_stop, min_departure_time, max_arrival_time, max_connections, include_earliest_arrival=False):
    stops, connections, trips, footpaths = timetable
    S = CSA(timetable, target_stop, min_departure_time, max_arrival_time, max_connections + 1, include_earliest_arrival)
    all_paths = sorted(extract_journeys(S, source_stop, min_departure_time, target_stop, changes=None), key=lambda journey: journey[0][2], reverse=True)
    for path_index, path in enumerate(all_paths):
        print("JOURNEY", path_index)
        journey = analyze_path(path, max_arrival_time, footpaths)
        print(str(journey))

In [464]:
paper_conn = [
    ("y", "t", 10, 11, "1"),
    ("z", "t", 9, 12, "2"),
    ("x", "t", 8, 13, "3"),
    ("x", "y", 8, 9, "4"),
    ("s", "z", 7, 8, "5"),
    ("s", "x", 6, 7, "6"),
    ("s", "t", 5, 14, "7")
]
timetable = (None, paper_conn, None, defaultdict(lambda: list()))

In [465]:
plan_route(timetable, "s", "t", 4, 15, max_connections=3, include_earliest_arrival=True)

JOURNEY 0
At 7, take the line 5 with arrival at time 8 in z
After reaching station z, change from line 5 to line 2 (estimated change time of 2 minutes)
At 9, take the line 2 with arrival at time 12 in t
JOURNEY 1
At 6, take the line 6 with arrival at time 7 in x
After reaching station x, change from line 6 to line 3 (estimated change time of 2 minutes)
At 8, take the line 3 with arrival at time 13 in t
JOURNEY 2
At 6, take the line 6 with arrival at time 7 in x
After reaching station x, change from line 6 to line 4 (estimated change time of 2 minutes)
At 8, take the line 4 with arrival at time 9 in y
After reaching station y, change from line 4 to line 1 (estimated change time of 2 minutes)
At 10, take the line 1 with arrival at time 11 in t
JOURNEY 3
At 5, take the line 7 with arrival at time 14 in t
